# meeting 122122

In [1]:
import pathlib

import pandas as pd
from pathlib import Path
import numpy as np
import os
from tTEM_toolbox import  process_well as pw
link = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\2022SS\ArcGIS\ttem_specs\shapefile\Usgs water well\MartsonTable6.xlsx')
import os
import numpy as np
import tTEM_toolbox as tt
import pandas as pd
import plotly.offline as po
import plotly.graph_objects as go
import plotly.express as px
from pyproj import Transformer
import pykrige as pk
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
from pathlib import Path


In [2]:
workdir = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test')
well_info = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah\USGSdownload\NWISMapperExport.xlsx'
location = r"C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\location.csv"
welllog = workdir.joinpath(r'Plot_with_well_log\Well_log.xlsx')
elevation = workdir.joinpath(r'well_Utah\usgs_water_elevation.csv')
ttemname = workdir.joinpath(r'Plot_with_well_log\PD1_I01_MOD.xyz')
ttemname2 = workdir.joinpath(r'Plot_with_well_log\PD22_I03_MOD.xyz')
ttemlsl_file = workdir.joinpath(r'Plot_with_well_log\lsll_I05_MOD.xyz')
DOI = workdir.joinpath(r'Plot_with_well_log\DOID1_DOIStaE.xyz')
time = '2022-3'
def splt_ttem(ttem_df, gwsurface_result):
    def get_distance(group1, group2):
        dis = np.sqrt((group1[0] - group2[0]) ** 2 + (group1[1] - group2[1]) ** 2)
        return dis
    abv_water_table = []
    blw_water_table = []
    ttem_groups = ttem_df.groupby(['UTMX', 'UTMY'])
    well_location = gwsurface_result[['UTMX', 'UTMY']].values
    for name, group in ttem_groups:
        ttem_xy = list(group[['UTMX', 'UTMY']].iloc[0])
        ttem_well_distance = list(map(lambda x: get_distance(ttem_xy, x), well_location))
        match = gwsurface_result.iloc[ttem_well_distance.index(min(ttem_well_distance))]
        elevation = match['water_elevation']
        ttem_abv = group[group['Elevation_End'] >= elevation]
        abv_water_table.append(ttem_abv)
        ttem_blw = group[group['Elevation_End'] < elevation]
        blw_water_table.append(ttem_blw)

    ttem_above = pd.concat(abv_water_table)
    ttem_below = pd.concat(blw_water_table)
    return ttem_above, ttem_below
def plot_bst(dataframe):
    """
    plot bootstrap result

    :param dataframe:
    :return: plotly fig
    """
    fig_hist = go.Figure()
    fig_hist.data = []
    fig_hist.add_trace(go.Histogram(x=dataframe.fine, name='Fine', marker_color='Blue', opacity=0.75))
    fig_hist.add_trace(go.Histogram(x=dataframe.coarse, name='Coarse', marker_color='Red', opacity=0.75))
    if dataframe.mix.sum() == 0:
        print("skip plot mix because there is no data")
    else:
        fig_hist.add_trace(go.Histogram(x=dataframe.mix, name='Mix', marker_color='Green', opacity=0.75))
    fig_hist.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

    return fig_hist
def data_fence(ttem_df,xmin,ymin,xmax,ymax):
    new_ttem_df = ttem_df[(ttem_df['UTMX']>xmin)&(ttem_df['UTMX']<xmax)&(ttem_df['UTMY']>ymin)&(ttem_df['UTMY']<ymax)]
    return new_ttem_df

# example north compare

In [3]:
ttem_n = tt.main.ProcessTTEM(ttem_path=[ttemname2],
                                   welllog=welllog,
                                   DOI_path=DOI,
                                    distance=1000
                                   layer_exclude=[],
                                    line_exclude=[])


Applying DOI |                                | 90/7968

Exclude layer []
Exclude line []
No point was excluded


Applying DOI |################################| 7968/7968


In [4]:
a = ttem_n.data()

In [ ]:
pure_north_data = data_fence(ttem_n.data(),348718,4208611,351419,4213504)


In [141]:
fig = go.Figure(tt.plot.generate_trace(pure_north_data,'ttem'))
fig.show(renderer='browser')

In [ ]:
water = tt.main.GWSurface(waterwell=well_info)

In [134]:
time = '2022-3'
water_format = water.format(elevation=None, time=time)
water_format13_3 =  water.format(elevation=None, time='2013-3')
water_format12_11 = water.format(elevation=None, time='2012-11')
elevation_diff12_13 = water_format12_11['sl_lev_va2012-11'].subtract(water_format13_3['sl_lev_va2013-3'])
water_format['water_elevation'] = water_format['sl_lev_va2022-3'] + elevation_diff12_13

In [136]:
ttem_above_wt, ttem_below_wt = splt_ttem(pure_north_data, water_format)

In [138]:
bootstrap_above = tt.main.ProcessTTEM(ttem_path=ttem_above_wt, welllog=welllog)
bootstrap_above_result = bootstrap_above.ttem_well_connect(distance=1000)
bootstrap_below = tt.main.ProcessTTEM(ttem_path=ttem_below_wt, welllog=welllog)
bootstrap_below_result = bootstrap_below.ttem_well_connect(distance=1000)

Reusing cached dataframe...
No layer was excluded
No line was excluded
No point was excluded
Will skip filterting DOI since no DOI file exist


Formating well logs |################################| 223/223
Selecting closest ttem trace |################################| 223/223
connecting ttem with well |################################| 6/6
C:\Users\jldz9\PycharmProjects\tTEM_toolbox\src\tTEM_toolbox\core\bootstrap.py:185: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



Total of 6 well with in radius (1000m), 217 skipped
Bootstraping...
Done!
Reusing cached dataframe...
No layer was excluded
No line was excluded
No point was excluded
Will skip filterting DOI since no DOI file exist


Formating well logs |################################| 223/223
Selecting closest ttem trace |################################| 223/223
connecting ttem with well |################################| 6/6
C:\Users\jldz9\PycharmProjects\tTEM_toolbox\src\tTEM_toolbox\core\bootstrap.py:185: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



Total of 6 well with in radius (1000m), 217 skipped
Bootstraping...
Done!


In [139]:
fig_hist_ab = plot_bst(bootstrap_above_result[1])
fig_hist_ab.show(renderer='browser')
fig_hist_bl = plot_bst(bootstrap_below_result[1])
fig_hist_bl.show(renderer='browser')

skip plot mix because there is no data


In [ ]:
import pathlib
isinstance(welllog,pathlib.PosixPath)

In [ ]:
ttem_n_limit = data_fence(ttem_n.data(),348569,4208615,351224,4213346)

In [ ]:
ttem_n = tt.main.ProcessTTEM(ttem_path=ttem_n_limit,
                                   welllog=welllog,
                                   DOI_path=DOI,
                                   layer_exclude=[],
                                    line_exclude=[])

In [6]:
ttem_n.ttem_well_connect(distance=1000)

Formating well logs |################################| 223/223
Selecting closest ttem trace |################################| 223/223
connecting ttem with well |###                             | 8/67

Total of 67 well with in radius (1000m), 156 skipped


connecting ttem with well |################################| 67/67
C:\Users\jldz9\PycharmProjects\tTEM_toolbox\src\tTEM_toolbox\core\bootstrap.py:186: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



Bootstraping...
Done!


(         ID  Line_No  Layer_No      UTMX       UTMY  Elevation_Cell  \
 0         1      100         1  340413.6  4197611.1         1742.79   
 1         1      100         2  340413.6  4197611.1         1741.75   
 2         1      100         3  340413.6  4197611.1         1740.61   
 3         1      100         4  340413.6  4197611.1         1739.37   
 4         1      100         5  340413.6  4197611.1         1738.03   
 ...     ...      ...       ...       ...        ...             ...   
 91332  4119      240         9  339113.6  4196442.1         1737.60   
 91333  4119      240        10  339113.6  4196442.1         1735.55   
 91334  4119      240        11  339113.6  4196442.1         1733.32   
 91335  4119      240        12  339113.6  4196442.1         1730.89   
 91336  4119      240        13  339113.6  4196442.1         1728.25   
 
        Resistivity  Resistivity_STD  Conductivity  Depth_top  Depth_bottom  \
 0            10.44            1.572       95.7854     

In [7]:
def resistivity_avg(ttem_df):
    new_1d_df = []
    coordinate_group = ttem_df.groupby(['UTMX','UTMY'])
    for name, group in coordinate_group:
        mean_resistivity = group['Resistivity'].mean()
        output_df = {'UTMX':name[0],'UTMY':name[1],'Mean_Resistivity':mean_resistivity}
        tmp_df = pd.DataFrame([output_df],index=None)
        new_1d_df.append(tmp_df)
    export_df = pd.concat(new_1d_df)
    return  export_df

In [3]:
ttemlsl = pd.read_fwf(ttemlsl_file, skiprows=34)
ttemlsl = ttemlsl.drop(columns="/")
ttemlsl = ttemlsl[ttemlsl.Thickness_STD != 9999]
ttemlsl["Elevation_End"] = ttemlsl["Elevation_Cell"].subtract(ttemlsl["Thickness"])
ttem_lsl = tt.main.ProcessTTEM(ttem_path=ttemlsl,
                                   welllog=welllog,

                                   layer_exclude=[],
                                    line_exclude=[])

Reusing cached dataframe...
Exclude layer []
Exclude line []
No point was excluded
Will skip filterting DOI since no DOI file exist


In [4]:
ttem_lsl_data = ttem_lsl.data()


In [8]:
ttem_lsl_output = resistivity_avg(ttem_lsl_data)


In [10]:
ttem_lsl_output.to_csv(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\ttem_little_salt_lake.csv')

In [ ]:
bootstrap_result=ttem_lsl.ttem_well_connect(distance=1000)

In [19]:
res_conf_df = tt.bootstrap.packup(bootstrap_result[1]['fine'],bootstrap_result[1]['mix'], bootstrap_result[1]['coarse'] )


In [13]:
rk_trans = tt.Rock_trans.rock_transform(bootstrap_result[0], res_conf_df)

In [23]:
res_conf_df['Fine_conf']

0    1.396658
1    3.435550
Name: Fine_conf, dtype: float64

In [18]:
rk_trans

,ID,Line_No,Layer_No,UTMX,UTMY,Elevation_Cell,Resistivity,Resistivity_STD,Conductivity,Depth_top,Depth_bottom,Thickness,Thickness_STD,Elevation_End,Identity,Identity_n
0,1,100,1,340413,4197616,1742.79,12.640,99.0,79.1139,0.00,1.0,1.00,99.0,1741.79,Coarse_grain,3
1,2,100,1,340413,4197619,1742.78,13.010,99.0,76.8639,0.00,1.0,1.00,99.0,1741.78,Coarse_grain,3
2,3,100,1,340414,4197622,1742.79,13.070,99.0,76.5111,0.00,1.0,1.00,99.0,1741.79,Coarse_grain,3
3,4,100,1,340415,4197625,1742.77,13.430,99.0,74.4602,0.00,1.0,1.00,99.0,1741.77,Coarse_grain,3
4,5,100,1,340417,4197627,1742.76,13.990,99.0,71.4796,0.00,1.0,1.00,99.0,1741.76,Coarse_grain,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79757,94,110,19,335017,4191667,1651.17,2.796,99.0,357.6540,101.06,120.0,18.94,99.0,1632.23,Fine_grain,1
79758,95,110,19,335016,4191676,1651.17,2.810,99.0,355.8720,101.06,120.0,18.94,99.0,1632.23,Fine_grain,1
79759,96,110,19,335014,4191685,1651.19,2.821,99.0,354.4840,101.06,120.0,18.94,99.0,1632.25,Fine_grain,1
79760,97,110,19,335013,4191692,1651.18,2.830,99.0,353.3570,101.06,120.0,18.94,99.0,1632.24,Fine_grain,1


In [15]:
def lithology_pct(rock_transform_df):
    """
    # Plan to group by the coordinate and generate a new dataframe with only [UTMX, UTMY]
    """
    new_1d_df = []
    coordinate_group = rock_transform_df.groupby(['UTMX','UTMY'])
    for name, group in coordinate_group:
        total_thickness_for_point = sum(group['Thickness'])
        lithology_group = group.groupby('Identity')
        try:
            fine_grain_thickness = lithology_group.get_group('Fine_grain')['Thickness'].sum()
        except KeyError:
            fine_grain_thickness = 0
        try:
            mixed_grain_thickness = lithology_group.get_group('Mix_grain')['Thickness'].sum()
        except KeyError:
            mixed_grain_thickness = 0
        try:
            coarse_grain_thickness = lithology_group.get_group('Coarse_grain')['Thickness'].sum()
        except KeyError:
            coarse_grain_thickness = 0
        tmp_df = pd.DataFrame([{'UTMX':name[0],
                  'UTMY':name[1],
                  'Fine grained ratio':fine_grain_thickness/float(total_thickness_for_point),
                  'mixed_grain_thickness':mixed_grain_thickness/float(total_thickness_for_point),
                  'coarse_grain_thickness':coarse_grain_thickness/float(total_thickness_for_point)}])

        new_1d_df.append(tmp_df)
    export = pd.concat(new_1d_df)
    return export

In [17]:
export = lithology_pct(rk_trans)
export.to_csv(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\lsl_rk.csv')

In [149]:
fig_rk = go.Figure(tt.plot.generate_trace(rk_trans,'rock') )
fig_rk.show(renderer='browser')

In [ ]:
fig_2 = go.Figure(tt.plot.generate_trace(ttemlsl,'ttem'))
trace = go.Figure(tt.plot.generate_trace(ttem_ct,'ttem'))
fig_2.add_trace(trace.data[0])
fig_2.show(renderer='browser')


In [ ]:
trace.data[0]

In [ ]:
fig1 = go.Figure(tt.plot.generate_trace(ttem.data(),'ttem'))
fig1.show(renderer='browser')

In [ ]:
trace2 = go.Scatter3d(x=ttemlsl.UTMX.values, y=ttemlsl.UTMY.values, z=ttemlsl.Elevation_Cell.values)
fig_3 = go.Figure(trace2)

In [ ]:
fig_3.data